In [70]:
import pandas as pd
from paretoset import paretoset
from utils import *
import numpy as np

In [71]:
results = pd.read_csv('results/pairwise_aln_nt/nt_x_ref_edited.csv')
ref_tobamo = list(parse_fasta('/home/tobamo/analize/project-tobamo/analysis/whole_orf/data/refs_tobamo_nt/tobamo_ref_genomes_57_.fasta').keys())
ref_virga = list(parse_fasta('/home/tobamo/analize/project-tobamo/analysis/whole_orf/data/refs_tobamo_nt/virgaviridae_RNA1_nt_20_.fasta').keys())

In [72]:
def calculate_pareto_set(group):
    group['on_pareto_front'] = paretoset(group[['identity_score', 'N/aln_len', 'gap_ratio']], sense=["max", "max", "min"])
    return group

In [73]:
results = results.groupby('contig_name', group_keys=False).apply(calculate_pareto_set)
pareto_true = results[results.on_pareto_front == True]

In [74]:
print(f"unique SRR: {pareto_true.SRR.nunique()}\nunique contigs: {pareto_true.contig_name.nunique()}")

unique SRR: 21
unique contigs: 77


In [75]:
pareto_true = pareto_true.sort_values(by=['SRR', 'contig_name', 'identity_score'], ascending=False).reset_index(drop=True)
pareto_true['ref_tax'] = np.where(pareto_true['ref_name'].isin(ref_tobamo), 'tobamo', 'virga')
# pareto_true.to_csv('results/pairwise_aln_nt/nt_pariwise_aln_pareto_true2.csv', index=False)

SRR metadata for Luka

In [76]:
srr_metadata = pd.read_csv('/home/tobamo/analize/tobamo/notebooks/srr_selection/combined_results.csv')
filtered_srr_metadata = srr_metadata[srr_metadata.run_id.isin(pareto_true.SRR.unique())]
sub = filtered_srr_metadata[['run_id', 'biosample_id', 'scientific_name', 'nickname', 'date', 'lng', 'lat']].drop_duplicates().reset_index(drop=True)
# sub.to_csv('77contigs_srr_metadata.csv')